In [ ]:
!git clone https://github.com/microsoft/UniPrompt.git

In [ ]:
%cd UniPrompt
!python3 -m pip install --upgrade pip setuptools
!python3 -m pip install .
%cd ..

In [9]:
config = {
    "experiment_details":"Ethos, gpt-4o, 0.7 temperature, 10 epochs, 7 batch size, 5 mini batch size, 1 iterations, 3 beam width, 2 group frequency, 0.5 epsilon",
    "dataset_path": "data/ethos.jsonl",
    "mini_batch_size": 5,
    "batch_size": 7,
    "iterations": 1,
    "epochs": 10,
    "logging_file_path": "logs/ethos.jsonl",
    "epsilon": 0.5,
    "beam_width": 3,
    "group_frequency": 2,
    "content_safety": True,
    "cache_path": "cache/ethos.db",
    "initial_prompt": "Given a query, you have to tell if it contains hate speech in any form or not.",
    "dataset_name": "ethos",
    "number_of_groups": 2,
    "metric_kwargs": {
        "type": "accuracy"
    },
    "solver_llm": {
        "model_kwargs": {
            "temperature": 0,
            "model": "gpt-4o"
        },
        "api_kwargs": {
            "api_type": "",
            "api_base": "",
            "api_version": "",
            "api_key": ""
        }
    },
    "expert_llm": {
        "model_kwargs": {
            "temperature": 0.7,
            "model": "gpt-4o"
        },
        "api_kwargs": {
            "api_type": "",
            "api_base": "",
            "api_version": "",
            "api_key": ""
        }
    },
    "grouping_llm": {
        "model_kwargs": {
            "temperature": 0.7,
            "model": "gpt-4o"
        },
        "api_kwargs": {
            "api_type": "",
            "api_base": "",
            "api_version": "",
            "api_key": ""
        }
    }
}

In [ ]:
from uniprompt.modular import baseUniprompt, load_dataset

In [ ]:
train_data, val_data, test_data = load_dataset(config["dataset_name"])

In [ ]:
uniprompt = baseUniprompt(config)
p = config["initial_prompt"]

In [ ]:
metrics = uniprompt.evaluate(data=test_data, prompt=p)
print(f"Metrics for initial prompt: {p}: {metrics}")

In [ ]:
uniprompt.initialize_candidates(initial_prompt = p, val_data=val_data)

In [ ]:
for epoch in range(config["epochs"]):
    # Create groups based on configured frequency
    p = uniprompt.get_best_prompt()
    if epoch % config["group_frequency"] == 0:
        # if you want to group every epoch then you can do that or you can group based on the grouping frequency set in config
        # grouping function is optional, you can provide your own grouping function
        uniprompt.create_groups(prompt=p, train_data=train_data)

    # Training iterations
    for _ in range(config["iterations"]):
        p = uniprompt.train(train_data=train_data, val_data=val_data)
        # evaluation function is optional, you can provide your own evaluation function
        metrics = uniprompt.evaluate(data=test_data, prompt=p)
        print(f"Epoch: {epoch}, Prompt: {p}, Metrics: {metrics}")


In [ ]:
final_metrics = uniprompt.evaluate(data=test_data, prompt=p)
print(f"Best prompt: {p}")
print(f"Final metrics: {final_metrics}")